## Objectives

- Explore and glean insights from a real dataset using pandas
- Practice using pandas for exploratory analysis, information gathering, and discovery
- Practice cleaning data and answering questions

## Dataset

You are to analyze campaign contributions to the 2016 U.S. presidential primary races made in California. Use the csv file located here: https://drive.google.com/file/d/1Lgg-PwXQ6TQLDowd6XyBxZw5g1NGWPjB/view?usp=sharing. You should download and save this file in the same folder as this notebook is stored.  This file originally came from the U.S. Federal Election Commission (https://www.fec.gov/).

**DO NOT PUSH THIS FILE TO YOUR GITHUB REPO!**

- Best practice is to not have DATA files in your code repo. As shown below, the default load is outside of the folder this notebook is in. If you change the folder where the file is stored please update the first cell!
- If you do accidentally push the file to your github repo - follow the directions here to fix it: https://docs.google.com/document/d/15Irgb5V5G7pKPWgAerH7FPMpKeQRunbNflaW-hR2hTA/edit?usp=sharing

Documentation for this data can be found here: https://drive.google.com/file/d/11o_SByceenv0NgNMstM-dxC1jL7I9fHL/view?usp=sharing

## Data Questions

You are working for a California state-wide election campaign. Your boss wants you to examine historic 2016 election contribution data to see what zipcodes are more supportive of fundraising for your candidate. 

Your boss asks you to filter out some of the records:
- Only use primary 2016 contribution data (more like how your race is).
- Concentrate on Bernie Sanders as a candidate (most a like your candidate)

The questions your boss wants answered is:
- Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount?
- What day(s) of the month do most people donate?

## Setup

Run the cell below as it will load the data into a pandas dataframe named `contrib`. Note that a custom date parser is defined to speed up loading. If Python were to guess the date format, it would take even longer to load.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# These commands below set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# Define a date parser to pass to read_csv
d = lambda x: datetime.strptime(x, '%d-%b-%y')

# Load the data
# We have this defaulted to the folder OUTSIDE of your repo - please change it as needed
contrib = pd.read_csv('/home/vanel/github/P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)

# Note - for now, it is okay to ignore the warning about mixed types. 

/tmp/ipykernel_134334/3373767128.py:14: DtypeWarning: Columns (6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  contrib = pd.read_csv('/home/vanel/github/P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)


***
## 1. Initial Data Checks (50 points)

First we will take a preliminary look at the data to check that it was loaded correctly and contains the info we need.

The questions to answer at the end of this section:
- Do we have the correct # of columns and rows. 
- Do the records contain data for the questions we want to answer 
- What columns are important? 
- What columns can be dropped?
- What are the data problems?

**1a.** Print the *shape* of the data. Does this match the expectation? (2 points)

In [2]:
# 1a YOUR CODE HERE
contrib.shape

(1125659, 18)

- **1a answer here:** 

Yes, the file contains 18 columns and 1,125,659 rows.

**1b.** Print a list of column names. Are all the columns included that are in the documentation? (2 points)

In [3]:
# 1b YOUR CODE HERE
contrib.columns

Index(['cmte_id', 'cand_id', 'cand_nm', 'contbr_nm', 'contbr_city',
       'contbr_st', 'contbr_zip', 'contbr_employer', 'contbr_occupation',
       'contb_receipt_amt', 'contb_receipt_dt', 'receipt_desc', 'memo_cd',
       'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp'],
      dtype='object')

- **1b answer here:** 

Yes, all the columns in the documentation are included.

**1c** Print out the first five rows of the dataset. How do the columns `cand_id`, `cand_nm` and `contbr_st` look? (3 points)

In [4]:
# 1c YOUR CODE HERE
contrib.head()

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00575795,P00003392,"Clinton, Hillary Rodham","AULL, ANNE",LARKSPUR,CA,"949,391,913.00",NaN,RETIRED,50.00,2016-04-26,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4768722,P2016
1,C00575795,P00003392,"Clinton, Hillary Rodham","CARROLL, MARYJEAN",CAMBRIA,CA,"934,284,638.00",NaN,RETIRED,200.00,2016-04-20,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4747242,P2016
2,C00575795,P00003392,"Clinton, Hillary Rodham","GANDARA, DESIREE",FONTANA,CA,"923,371,507.00",NaN,RETIRED,5.00,2016-04-02,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4666603,P2016
3,C00577130,P60007168,"Sanders, Bernard","LEE, ALAN",CAMARILLO,CA,"930,111,214.00",AT&T GOVERNMENT SOLUTIONS,SOFTWARE ENGINEER,40.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKWA097,P2016
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,"902,784,310.00",VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016


- **1c answer here:** 

`cand_id`: consists of a unique identification nummber that follows P. With P00003392 being for Clinton and P60007168 being for Sanders.

`cand_nm`: includes Hilary Clinton and Bernie Sanders.

`contbr_st`: The first 5 rows show contribution state as California.

**1d.** Print out the values for the column `election_tp`. In your own words, based on the documentation, what information does the `election_tp` variable contain? Do the values in the column match the documentation? (3 points)

In [5]:
# 1d YOUR CODE HERE
contrib['election_tp'].unique()

array(['P2016', 'G2016', nan, 'P2020'], dtype=object)

- **1d answer here:** 

`election_tp`: the variable contains values with the first letter P or G indicating Primary or General elections followed by the year. So P2016 would be contributions made for the 2016 Primary elections. It also seem to contain some null values but other than that, the values match the documentation.

**1e.** Print out the datatypes for all of the columns. What are the datatypes for the `contbr_zip`, `contb_receipt_amt`, `contb_receipt_dt`? (5 points)

In [6]:
# 1e YOUR CODE HERE
contrib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125659 entries, 0 to 1125658
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   cmte_id            1125659 non-null  object        
 1   cand_id            1125659 non-null  object        
 2   cand_nm            1125659 non-null  object        
 3   contbr_nm          1125659 non-null  object        
 4   contbr_city        1125633 non-null  object        
 5   contbr_st          1125659 non-null  object        
 6   contbr_zip         1125564 non-null  object        
 7   contbr_employer    967757 non-null   object        
 8   contbr_occupation  1115260 non-null  object        
 9   contb_receipt_amt  1125659 non-null  float64       
 10  contb_receipt_dt   1125659 non-null  datetime64[ns]
 11  receipt_desc       15045 non-null    object        
 12  memo_cd            144268 non-null   object        
 13  memo_text          501148 n

- **1e answer here:** 

`contbr_zip`: object/string

`contb_receipt_amt`: float64

`contb_receipt_dt`: datetime64[ns]

**1f.** What columns have the most non-nulls?  Would you recommend to drop any columns based on the number of nulls? (5 points)

In [7]:
# 1f YOUR CODE HERE
# Get the non-null value counts for each column
non_null_counts = contrib.notnull().sum()

# Sort the non-null value counts in descending order
non_null_counts_sorted = non_null_counts.sort_values(ascending=False)

# Print the columns with the most non-null values
print("Columns with the most non-null values:")
print(non_null_counts_sorted)

#all nulls
contrib.isnull().sum()

Columns with the most non-null values:
cmte_id              1125659
cand_id              1125659
tran_id              1125659
file_num             1125659
form_tp              1125659
contb_receipt_dt     1125659
contb_receipt_amt    1125659
contbr_st            1125659
contbr_nm            1125659
cand_nm              1125659
contbr_city          1125633
contbr_zip           1125564
election_tp          1124234
contbr_occupation    1115260
contbr_employer       967757
memo_text             501148
memo_cd               144268
receipt_desc           15045
dtype: int64


cmte_id                    0
cand_id                    0
cand_nm                    0
contbr_nm                  0
contbr_city               26
contbr_st                  0
contbr_zip                95
contbr_employer       157902
contbr_occupation      10399
contb_receipt_amt          0
contb_receipt_dt           0
receipt_desc         1110614
memo_cd               981391
memo_text             624511
form_tp                    0
file_num                   0
tran_id                    0
election_tp             1425
dtype: int64

- **1f answer here:** 

['cmte_id', 'cand_id', 'cand_nm', 'file_num', 'tran_id','contbr_nm', 'contb_receipt_amt', 'contb_receipt_dt','contbr_st', 
'contbr_city'] have the most non-null values. 

Given the basis of the data question, I would recommend dropping ['contbr_employer','receipt_desc','memo_cd','memo_text'] not only due to the large number of null values but also due to our main question of interest which is to find which zip codes had the highest count and contribution amounts. 

**1g.** A column we know that we want to use is the cand_nm column.  From the documentation each candidate is a unique candidate id also. Check data quality of `cand_id` column to see if it matches `cand_nm` column. Specifically check to ensure our targetted candidate 'Bernard Sanders' always has the same cand_id throughout. Any issues with `cand_nm` matching `cand_id`? (5 points)

In [8]:
# 1g YOUR CODE HERE

#CHECKING ALL CANDIDATE IDS
# Group the data by candidate name and candidate ID
candidate_groups = contrib.groupby(['cand_nm', 'cand_id'])

# List the unique candidate IDs for each candidate name
unique_candidates = candidate_groups.size().reset_index(name='count')

# Filter candidates with multiple candidate IDs
candidates_with_multiple_ids = unique_candidates[unique_candidates['count'] > 1]

# Print the candidate names and their associated candidate IDs
for index, row in candidates_with_multiple_ids.iterrows():
    candidate_name = row['cand_nm']
    candidate_id = row['cand_id']
    print(f"Candidate: {candidate_name}, Candidate ID: {candidate_id}")

#CHECKING FOR BERNARD SANDERS
# Step 1: Filter the DataFrame to include only rows where 'cand_nm' is 'Bernard Sanders'
sanders_contributions = contrib[contrib['cand_nm'] == 'Sanders, Bernard']

# Step 2: Check unique 'cand_id' values for 'Bernard Sanders'
unique_cand_ids = sanders_contributions['cand_id'].unique()

# Step 3: Check if there are multiple 'cand_id' values associated with 'Bernard Sanders'
if len(unique_cand_ids) == 1:
    print("\nData quality check passed: 'Bernard Sanders' has a consistent 'cand_id'.")
    print("Unique cand_id for 'Bernard Sanders':", unique_cand_ids[0])
else:
    print("\nData quality check failed: 'Bernard Sanders' has multiple cand_ids.")
    print("Unique cand_ids for 'Bernard Sanders':", unique_cand_ids)


Candidate: Bush, Jeb, Candidate ID: P60008059
Candidate: Carson, Benjamin S., Candidate ID: P60005915
Candidate: Christie, Christopher J., Candidate ID: P60008521
Candidate: Clinton, Hillary Rodham, Candidate ID: P00003392
Candidate: Cruz, Rafael Edward 'Ted', Candidate ID: P60006111
Candidate: Fiorina, Carly, Candidate ID: P60007242
Candidate: Gilmore, James S III, Candidate ID: P80003379
Candidate: Graham, Lindsey O., Candidate ID: P60007697
Candidate: Huckabee, Mike, Candidate ID: P80003478
Candidate: Jindal, Bobby, Candidate ID: P60008398
Candidate: Johnson, Gary, Candidate ID: P20002671
Candidate: Kasich, John R., Candidate ID: P60003670
Candidate: Lessig, Lawrence, Candidate ID: P60009685
Candidate: McMullin, Evan, Candidate ID: P60022654
Candidate: O'Malley, Martin Joseph, Candidate ID: P60007671
Candidate: Pataki, George E., Candidate ID: P60007572
Candidate: Paul, Rand, Candidate ID: P40003576
Candidate: Perry, James R. (Rick), Candidate ID: P20003281
Candidate: Rubio, Marco, 

- **1g answer here:** 

Each candidate has a unique `cand_id` and Bernard Sanders has a unique `cand_id`  of P60007168

**1h.** Another area to check is to make sure all of the records are from California. Check the `contbr_st` column - are there any records outside of California based on `contbr_st`? (5 points)

In [9]:
# 1h YOUR CODE HERE
# Check unique values in the 'contbr_st' column
unique_states = contrib['contbr_st'].unique()
# Check if 'CA' is the only unique value (indicating all records are from California)
if len(unique_states) == 1 and unique_states[0] == 'CA':
    print("Data quality check passed: All records are from California.")
else:
    print("Data quality check failed: Records are not exclusively from California.")
    print("Unique states in the dataset:", unique_states)

Data quality check passed: All records are from California.


- **1h answer here:** 

Data quality check passed: All records are from California.

**1i.** The next column to check for the analysis is the `tran_id` column. This column could be the primary key so look for duplicates. How many duplicate entries are there? Any pattern for why are there duplicate entries? (5 points)

In [10]:
# 1i YOUR CODE HERE
# Check for duplicates in the 'tran_id' column
duplicates = contrib[contrib.duplicated(subset='tran_id', keep=False)]

# Count the number of duplicate entries in the 'tran_id' column
duplicate_counts = contrib['tran_id'].duplicated().sum()

print("Number of duplicate entries in the 'tran_id' column:", duplicate_counts)

# Print the DataFrame containing duplicate entries
print(duplicates)

#save duplicates to file for further inspection
#duplicates.to_csv('duplicate_entries.csv', index=False)


Number of duplicate entries in the 'tran_id' column: 3454
           cmte_id    cand_id                    cand_nm  \
57       C00575795  P00003392    Clinton, Hillary Rodham   
90       C00575795  P00003392    Clinton, Hillary Rodham   
91       C00575795  P00003392    Clinton, Hillary Rodham   
174      C00574624  P60006111  Cruz, Rafael Edward 'Ted'   
175      C00574624  P60006111  Cruz, Rafael Edward 'Ted'   
...            ...        ...                        ...   
1125589  C00573519  P60005915        Carson, Benjamin S.   
1125596  C00573519  P60005915        Carson, Benjamin S.   
1125597  C00573519  P60005915        Carson, Benjamin S.   
1125598  C00573519  P60005915        Carson, Benjamin S.   
1125614  C00573519  P60005915        Carson, Benjamin S.   

                     contbr_nm    contbr_city contbr_st     contbr_zip  \
57             SANMUKH, BHAKTA      SAN DIMAS        CA 917,733,736.00   
90             ROBERT, MONKMAN        FREMONT        CA 945,365,810.00   

- **1i answer here:** 

There are 3454 duplicate entries. Upon further inspection, the entries with duplicated `train_id` were made by the same people, on the same day and to the same candidates, same city and all. What seemed to cause duplicate `train_id` comes from the `contbr_employer`, `contbr_occupation`columns wirh the second duplicate contribution described as 'INFORMATION REQUESTED', with the `memo_text` being "* HILLARY VICTORY FUND"

**1j.** Another column to check is the `contb_receipt_amt` that shows the donation amounts. How many negative donations are included? What do negative donations mean? Please show at least pull a few rows to look at the records with negative donations. Do these records match with the expectation of why a negative donation would happen? (5 points)

In [11]:
# 1j YOUR CODE HERE
# Filter the DataFrame to include only rows with negative donation amounts
negative_donations = contrib[contrib['contb_receipt_amt'] < 0]

# Count the number of negative donations
num_negative_donations = negative_donations.shape[0]
print("Number of negative donations:", num_negative_donations)

# Show a few rows of negative donation records
if num_negative_donations > 0:
    print("\nSample of records with negative donations:")
    print(negative_donations.head())

#negative_donations.to_csv('negative_entries.csv', index=False)

Number of negative donations: 11896

Sample of records with negative donations:
       cmte_id    cand_id                    cand_nm           contbr_nm  \
19   C00574624  P60006111  Cruz, Rafael Edward 'Ted'    JOLLIFF, RICHARD   
23   C00574624  P60006111  Cruz, Rafael Edward 'Ted'    JOLLIFF, RICHARD   
81   C00574624  P60006111  Cruz, Rafael Edward 'Ted'    JOLLIFF, RICHARD   
190  C00574624  P60006111  Cruz, Rafael Edward 'Ted'    NOWELL, DIANA L.   
213  C00574624  P60006111  Cruz, Rafael Edward 'Ted'  LICHTY, ANDREW MR.   

                contbr_city contbr_st     contbr_zip  \
19                    CHICO        CA 959,289,507.00   
23                    CHICO        CA 959,289,507.00   
81                    CHICO        CA 959,289,507.00   
190  RANCHO SANTA MARGARITA        CA 926,884,928.00   
213               SAN DIEGO        CA 921,096,720.00   

                        contbr_employer     contbr_occupation  \
19                        SELF EMPLOYED               RANCHER

- **1j answer here:**

There are 11,896 negative donations. From the `receipt_desc` and `memo_text` the negative contributions seem to be from either refunds, REDESIGNATION TO GENERAL or PRESIDENTIAL or REATTRIBUTION TO SPOUSE or another candidate. The number of records match the with the expectation of why a negative donation would happen.

**1k.** One more column to look at is the date of donation column. Are there any dates outside of the primary period (defined as 1 Jan 2014 to 7 June 2016)? Are the dates well-formatted for our analysis? (5 points)

In [12]:
# 1k YOUR CODE HERE

# Define the primary period
primary_period_start = pd.to_datetime('2014-01-01')
primary_period_end = pd.to_datetime('2016-06-07')

# Filter the DataFrame to include only dates outside the primary period
outside_primary_period = contrib[(contrib['contb_receipt_dt'] < primary_period_start) | 
                                 (contrib['contb_receipt_dt'] > primary_period_end)]

# Count the number of dates outside the primary period
num_outside_primary_period = outside_primary_period.shape[0]

print("Number of donations outside the primary period:", num_outside_primary_period)

# Check the formatting of dates
date_format_correct = contrib['contb_receipt_dt'].notnull().all()

if date_format_correct:
    print("Dates are well-formatted for analysis.")
else:
    print("There are formatting issues with some dates.")

# Show a sample of records with dates outside the primary period
if num_outside_primary_period > 0:
    print("\nSample of records with dates outside the primary period:")
    print(outside_primary_period.head())

Number of donations outside the primary period: 451372
Dates are well-formatted for analysis.

Sample of records with dates outside the primary period:
         cmte_id    cand_id                    cand_nm          contbr_nm  \
9932   C00458844  P60006723               Rubio, Marco  WHEELER, MARY MS.   
9994   C00458844  P60006723               Rubio, Marco  WHEELER, MARY MS.   
14673  C00574624  P60006111  Cruz, Rafael Edward 'Ted'    RANDALL, DICK J   
14682  C00605568  P20002671              Johnson, Gary          BOAL, ROB   
14697  C00605568  P20002671              Johnson, Gary        LEE, JASCHA   

      contbr_city contbr_st     contbr_zip contbr_employer  contbr_occupation  \
9932     ATHERTON        CA 940,273,415.00   SELF-EMPLOYED  INTERIOR DESIGNER   
9994     ATHERTON        CA 940,273,415.00   SELF-EMPLOYED  INTERIOR DESIGNER   
14673   CUPERTINO        CA      95,014.00             NaN                NaN   
14682    OAK PARK        CA      91,377.00         VENDAVO   

- **1k answer here:**

Number of donations outside the primary period are 451372 and the Dates are well-formatted for analysis.

**1l.** Finally, answer the initial questions in the cells below (5 points)

**1l.1** Do we have the correct # of columns and rows.

- **1l.1 answer here:**

Yes. We have 18 columns and 1125659 rows.

**1l.2** Do the records contain data for the questions we want to answer?

- **1l.2 answer here:**

Yes. The data contains information to answer what zipcodes had the highest contribution count and amount and we can extract what days of the month do most people donate from the 'contb_receipt_dt' column

**1l.3** What columns are important?

- **1l.3 answer here:** 

`cmte_id`,`cand_id`, `cand_nm`, `contbr_city`,`contbr_st`,`contbr_zip`,`contb_receipt_amt`, `contb_receipt_dt` ,`form_tp`,`file_num`,`tran_id `, `election_tp`

**1l.4** What columns can be dropped?

- **1l.4 answer here:** 

`contbr_nm`, `contbr_employer`, `contbr_occupation`, `receipt_desc`, `memo_cd`, `memo_text`

**1l.5** What are the data problems?

- **1l.5 answer here:**

Potential data problems to consider:
1. Missing or incomplete values in the relevant columns (`contb_city`, `contbr_zip`).
2. Data entry errors or inconsistencies in the `contbr_zip` column (e.g., invalid or non-standard zip codes).
3. Date formatting issues in the `contb_receipt_dt` column.
4. Duplicates in the data (e.g., duplicate transactions or contributions).
5. Outliers or unusual values in the `contb_receipt_amt` column.
6. Inconsistent or unexpected data patterns.

**1l.6** List any assumptions so far:

- **1l.6 answer here:**

My assumptions are: 

that the provided dataset contains accurate and comprehensive information about contributions made during the 2016 primary election period.

that the `contb_receipt_dt` column represents the actual date of contribution.

that the `contb_receipt_amt` column represents the actual amount of contribution.

that the `contbr_zip` column represents valid 5-digit zip codes.

that there are no significant data quality issues that would severely impact our analysis. However, I acknowledge that further data exploration and cleaning may be necessary to address potential issues.

***
## 2. Data filtering and data quality fixes (30 points)

Now that we have a basic understanding of the data, let's filter out the records we don't need and fix the data.

**2a.** From the dataset filter out (remove) any election_tp not in the primary election. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [13]:
# 2a YOUR CODE HERE

# Filter out rows with election_tp not in the primary election
contrib = contrib[contrib['election_tp'] == 'P2016']

# Print the shape of the dataframe after filtering
print("Shape of the dataframe after filtering:", contrib.shape)

Shape of the dataframe after filtering: (810481, 18)


**2b.** From the dataset filter out (remove) any candidate that is not Bernie Sanders. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [14]:
# 2b YOUR CODE HERE

# Filter out rows where the candidate is not Bernie Sanders
contrib = contrib[contrib['cand_nm'] == 'Sanders, Bernard']

# Print the shape of the dataframe after filtering
print("Shape of the dataframe after filtering:", contrib.shape)

Shape of the dataframe after filtering: (407171, 18)


**2c.** The `contbr_zip` column is not formatted well for our analysis. Make a new zipcode column that is the five-digit zipcodes. Filter out any records outside of California based on the zipcode. Print/show the shape of the dataframe after the filtering is complete. (10 points).

- You will have to research what the valid 5-digit zipcodes for California are!

In [15]:
# 2c YOUR CODE HERE
# Pad shorter values with zeros and truncate longer values to 5 digits
contrib['zipcode'] = contrib['contbr_zip'].astype(str).str.ljust(5, '0').str[:5]

# Filter out records with zipcodes outside of California
contrib = contrib[contrib['zipcode'].str.startswith('9')]

# Print the shape of the dataframe after filtering
print("Shape of the dataframe after filtering:", contrib.shape)

Shape of the dataframe after filtering: (407161, 19)


**2d.** The receipt amount column has negative donations. After talking with your team, a decision was made that the best course of action is to remove these negative values so that the donation count and amount is more accurate. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [16]:
# 2d YOUR CODE HERE

# Filter out rows with negative donation amounts
contrib = contrib[contrib['contb_receipt_amt'] >= 0]

# Print the shape of the dataframe after filtering
print("Shape of the dataframe after filtering:", contrib.shape)

Shape of the dataframe after filtering: (404073, 19)


**2e.** From the dataset drop any columns that won't be used in the analysis. Print/show the shape of the dataframe after the dropping is complete. What columns did you drop and why? (5 points)

In [17]:
# 2e YOUR CODE HERE
columns_to_drop = ['cmte_id','cand_id', 'cand_nm','contbr_nm', 'contbr_city', 'contbr_st','contbr_zip', 'contbr_employer', 'contbr_occupation', 'receipt_desc', 'memo_cd', 
                   'memo_text','form_tp','file_num', 'tran_id','election_tp']

# Drop the columns
contrib = contrib.drop(columns=columns_to_drop)

# Print the shape of the dataframe after dropping columns
print("Shape of the dataframe after dropping columns:", contrib.shape)

Shape of the dataframe after dropping columns: (404073, 3)


- **2e answer here:**

A total of 16 columns were dropped.

Columns that were dropped: 

`cmte_id` There is only 1 unique committee ID for our focus candidate Bernard Sanders which is C00577130.

`cand_id`, `cand_nm` only 1 unique `cand_id` (P60007168) and `cand_nm` (Sanders, Bernard)

`contbr_nm`, `contbr_employer`, `contbr_occupation`, `receipt_desc`, `memo_cd`, `memo_text`, `form_tp`,`file_num`, `tran_id`, because they weren't neessary for the analysis. 

`contbr_city`,`contbr_st`,`contbr_zip` were also dropped because the new column `zipcode` encapsulates all of this information. 

`election_tp` Our data frame was filtered to only focus on the primary 2016. So this column wouldn't be used for our analysis further.

**2f.** List any assumptions that you made up to this point:

- **2f answer here:**

My assumptions are:

1. that the date formatting, zipcode formatting, and other data formats are consistent and suitable for analysis.
2. that filtering contributions based on California zipcodes will focus our analysis on contributions coming from California residents, aligning with the objectives of a California state-wide election campaign.
3. that the cleaned data is accurate and free from significant errors or inconsistencies. So, we should now be able to answer the questions.

***
## 3. Answering the questions (20 points)

Now that the data is cleaned and filterd - let's answer the two questions from your boss!

**3a.** Which zipcode had the highest count of contributions and the most dollar amount? (10 points)

In [19]:
# 3a YOUR CODE HERE
# Group the data by zipcode and calculate count and sum
zipcode_stats = contrib.groupby('zipcode').agg({'contb_receipt_amt': ['count', 'sum']})

# Rename columns for clarity
zipcode_stats.columns = ['contribution_count', 'total_donation_amount']

# Find the zipcode with the highest count of contributions
zipcode_max_count = zipcode_stats['contribution_count'].idxmax()
max_count = zipcode_stats.loc[zipcode_max_count, 'contribution_count']

# Find the zipcode with the highest sum of donation amounts
zipcode_max_amount = zipcode_stats['total_donation_amount'].idxmax()
max_amount = zipcode_stats.loc[zipcode_max_amount, 'total_donation_amount']

print("Zipcode with the highest count of contributions:", zipcode_max_count)
print("Count of contributions:", max_count)

print("\nZipcode with the highest total donation amount:", zipcode_max_amount)
print(f"Total donation amount: ${max_amount:,.2f}")

Zipcode with the highest count of contributions: 94110
Count of contributions: 4046

Zipcode with the highest total donation amount: 94110
Total donation amount: $294,061.13


- **3a answer here:** 

Zipcode with the highest count of contributions and most dollar amount: 94110
    
    count of contribution for 94110: 4046
    
    dollar donation amount for 94110: $294,061.13

**3b.** What day(s) of the month do most people donate? (10 points)

In [20]:
# 3b YOUR CODE HERE
# Convert 'contb_receipt_dt' to datetime format
contrib['contb_receipt_dt'] = pd.to_datetime(contrib['contb_receipt_dt'])

# Extract day of the month
contrib['day_of_month'] = contrib['contb_receipt_dt'].dt.day

# Count donations for each day of the month
donation_counts_per_day = contrib['day_of_month'].value_counts()

# Find the day(s) with the highest number of donations
max_donation_day = donation_counts_per_day.idxmax()
max_donation_count = donation_counts_per_day.max()

print("Day of the month with the most donations:", max_donation_day)
print("\nNumber of donations on the most active day:", max_donation_count)

print("\nDays with the top 5 donations")
print(donation_counts_per_day.head())

Day of the month with the most donations: 29

Number of donations on the most active day: 23511

Days with the top 5 donations
29    23511
30    22902
31    19488
14    17793
8     16774
Name: day_of_month, dtype: int64


- **3b answer here:** 

Most people donate towards the end of the month with 29th being the day that receives the most donations. Followed by 30th and 31st.
1. 29th > 23511 donations
2. 30th > 22902 donations
3. 31st > 19488 donations

## If you have feedback for this homework, please submit it using the link below:

http://goo.gl/forms/74yCiQTf6k